In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import sys

In [2]:
# The test hyper-parameter, default is argv_list = []
argv_list = ['complex', 'abstract']
#argv_list = []

In [3]:
# load the data from text
data_total = pd.read_csv('../Gene_DATA/sourcePathway.txt')

# extract the info based on regex
Namelist = data_total['TXT'].str.extract(r'^(?P<GenomeType>[a-z]+)\s(?P<GenomeName>[\w\/\-()+]+)$').dropna(axis=0).reset_index(drop=True)
Edgelist = data_total['TXT'].str.extract(r'^(?P<edgeStart>[\w\/\-()+]+)\s(?P<edgeEnd>[\w\/\-()+]+)\s(?P<edgeType>[\w\>\|-]+)$').dropna(axis=0).reset_index(drop=True)

In [4]:
if len(argv_list):
    # Here we started to choose to discard some features/node types 
    le = preprocessing.LabelEncoder()
    le.fit(Namelist['GenomeType'])
    all_node_class = list(le.classes_)

    # Input error check
    for elem in argv_list:
        if elem not in all_node_class:
            raise NameError('The input argument {} is not defined in node class.'.format(elem))
            sys.exit()

    # Start to exclude the node type
    Namelist = Namelist[~Namelist['GenomeType'].isin(argv_list)].reset_index(drop=True)

In [5]:
# The beforehand check
# Here we check the edge that does not showup in node list and drop these edges
Namelist_l = list(Namelist['GenomeName'])
Edgelist_l = list(Edgelist.iloc[:,0].values)
Edgelist_ll = list(Edgelist.iloc[:,1].values)
exclude_list = []
for idx, (elem, elem2) in enumerate(zip(Edgelist_l, Edgelist_ll)):
        if ((elem not in Namelist_l) or (elem2 not in Namelist_l)):
            exclude_list.append(idx)

Edgelist = Edgelist.drop(exclude_list).reset_index(drop=True)

In [6]:
# Label the index of each edge
# Make Edgelist --> edge_index

# Label edge_index
le = preprocessing.LabelEncoder()
le.fit(Namelist['GenomeName'])
edge_index = le.transform(Edgelist.iloc[:,:2].values.reshape(-1)).reshape(-1,2)

# Label edge_class
le = preprocessing.LabelEncoder()
le.fit(Edgelist['edgeType'])
edge_class = le.transform(Edgelist['edgeType'])

# Label node class
le = preprocessing.LabelEncoder()
le.fit(Namelist['GenomeType'])
node_class = le.transform(Namelist['GenomeType'])

# The combination of node info
node_all = np.concatenate([Namelist['GenomeName'].values.reshape(-1,1), node_class.reshape(-1,1)], axis=1)

In [9]:
node_all

array([['1/2-SBSRNA4', 1],
       ['AADAT', 1],
       ['AAGAB', 1],
       ...,
       ['PTRF', 1],
       ['NFIX', 1],
       ['SELP', 1]], dtype=object)